# <center> AI4LR - Final Assignment [100 Points]

In [44]:
# IMPORTANT
Your_name = "Aditya R"
Your_emailid = "adiramachandran59@gmail.com"

---

## Dataset
- Add [this dataset](https://www.kaggle.com/datasets/romainpessia/artificial-lunar-rocky-landscape-dataset) to your Kaggle notebook.

## Instructions for this project
- Don't forget to turn on GPU for training your model(s).
- Without changing anything in the notebook if you run it then you will get the val_iou_score of around 0.20.
- Your goal of this project is to increase this val_iou_score as much as you can.
- Evaluation of this project will be based on your best acquired val_iou_score seen in the notebook.
- Your val_iou_score will be the percentage you will receive for this project. 
- If your best val_iou_score is 0.41 then you will score 41/100 points in this project.
- Try to avoid any errors before submitting your notebook.

## Tips to increase the performance of your model
- Increase the number of epochs.
- Increase the number of layers in your model.
- Using SOTA high performance networks with transfer learning.
- Using callbacks and carefully observing your model performance.
- You can use the methods taught to you in this training program or any other methods of your own choice to increase the performance!

## Guidelines on making changes to this notebook
**1)** Add a descriptive comment to your code for whatever changes you are making in this notebook.
- For example, if you are adding an extra Conv2D layer, write about all the aspects of the Conv2D layer you are adding.
- The commnt should be placed at the point where the layer will be added.

**2)** Show model properties before and after the changes were made.
- For example, if you changed the layers - added, deleted, e.t.c.

**3)** If you use new data preprocessing techniques that are not already part of this notebook, you must explain their inner workings using markdown cells.
- Without this explanation, your techniques will not be considered for evaluation.
- Use texts and images to explain this process.

**4)** Make use of tables and plots that contributed to the improvement of your model.
- Assume if increasing the epochs and decreasing the learning rate contributed in the improvement of your model.
- You will first show these improvements using plots of val_iou_scores vs epochs as well as val_iou_scores vs learning rate.0
- Then make use of tables to show iou scores for different learning rates.
- For example, table 1 for lr_1 to show iou values for epochs 30 t0 50, table 2 to show iou values from epochs 30 to 50, and so on.
- It is therefore advised to work on one improvement, optimize it, plot it, document it, then proceed to the next improvement - till you get a satisfactory IOU score.

**5)** Final improvement summary table.
- Prepare a table with columns (changes, improvments description, increase in iou from, increase in iou to)
- List out all the changes you made to improve your final model performance.

## Coding for this project

In [45]:
!pip install -q segmentation_models

In [46]:
# import the necessary Library
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split

* Provide environment variable SM_FRAMEWORK=keras / SM_FRAMEWORK=tf.keras before import segmentation_models
* Change framework sm.set_framework('keras') / sm.set_framework('tf.keras')

In [47]:
# Setting framework environment
os.environ["SM_FRAMEWORK"] = "tf.keras"
sm.set_framework('tf.keras')
keras.backend.set_image_data_format('channels_last')

In [48]:
# To hide warnings
import warnings
warnings.filterwarnings("ignore")

## Data Preprocessing Pipeline

In [50]:
H = 480 # height of image, has been changed to 480 from 256, since reducing the size of the original image would result in lower amount of features for the model to train on, hence a lower score.
W = 480 # width of image, has been changed to 480 from 256, since reducing the size of the original image would result in lower amount of features for the model to train on, hence a lower score. Could not be able to be changed to 720 since VGG16 models accept only square shaped images as input.

'''This function is used to return the list of path for images and masks in
sorted order from the given directory respectively.'''
# function to return list of image paths and mask paths 
def process_data(IMG_DIR, MASK_DIR):
    images = [os.path.join(IMG_DIR, x) for x in sorted(os.listdir(IMG_DIR))]
    masks = [os.path.join(MASK_DIR, x) for x in sorted(os.listdir(MASK_DIR))]

    return images, masks

'''This function is used to return splitted list of images and corresponding 
mask paths in train and test by providing test size.'''
# function to load data and train test split
def load_data(IMG_DIR, MASK_DIR):
    X, y = process_data(IMG_DIR, MASK_DIR)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

'''This function is used to read images. It takes image path as input. 
After reading image it is resized by width and height provide above(256 x 256). 
Next normalization is done by dividing each values with 255. And the result is returned.'''
# function to read image
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

'''This function is used to read masks.'''
# function to read mask
def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.int32)
    return x

'''This function is used to generate tensorflow data pipeline. 
The tensorflow data pipeline is mapped to function ‘preprocess’ .'''
# function for tensorflow dataset pipeline
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)
    return dataset

'''This function takes image and mask path. 
It reads the image and mask as provided by paths. 
Mask is one hot encoded for multi class segmentation (here 4 class).'''
# function to read image and mask amd create one hot encoding for mask
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image = read_image(x)
        mask = read_mask(y)

        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, 4, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 4])

    return image, mask

## Load the dataset

In [51]:
'''RENDER_IMAGE_DIR_PATH: ‘Path of image directory’
GROUND_MASK_DIR_PATH: ‘Path of mask directory’

Here load_data function is called. This will load the dataset paths and 
split it into X_train, X_test, y_train, y_test '''

RENDER_IMAGE_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/render'
GROUND_MASK_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/clean'

X_train, X_test, y_train, y_test = load_data(RENDER_IMAGE_DIR_PATH, GROUND_MASK_DIR_PATH)
print(f"Dataset:\n Train: {len(X_train)} \n Test: {len(X_test)}")

Dataset:
 Train: 7812 
 Test: 1954


## Generate tensorflow data pipeline

In [52]:
batch_size = 16 # Changed the batch size to 16 to match the hyperparameter provided to the model

'''Here the tf_dataset function is called will generate the tensorflow data pipeline.'''
# calling tf_dataset
train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
valid_dataset = tf_dataset(X_test, y_test, batch=batch_size)

## Creating U-net Architecture

In [53]:
# Creating U-Net Architecture from scratch was resulting in lower val_iou_socres

# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
# from tensorflow.keras.models import Model

# '''conv_block it is used to create one block with two convolution layer 
# followed by BatchNormalization and activation function relu. 
# If the pooling is required then Maxpool2D is applied and return it else not.'''
# # function to create convolution block
# def conv_block(inputs, filters, pool=True):
#     x = Conv2D(filters, 3, padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     x = Conv2D(filters, 3, padding="same")(x)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     if pool == True:
#         p = MaxPool2D((2, 2))(x)
#         return x, p
#     else:
#         return x

# '''build_unet it is used to create the U-net architecture.'''
# # function to build U-net
# def build_unet(shape, num_classes):
#     inputs = Input(shape)

#     """ Encoder """
#     x1, p1 = conv_block(inputs, 16, pool=True)
#     x2, p2 = conv_block(p1, 32, pool=True)
#     x3, p3 = conv_block(p2, 48, pool=True)
#     x4, p4 = conv_block(p3, 64, pool=True)

#     """ Bridge """
#     b1 = conv_block(p4, 128, pool=False)

#     """ Decoder """
#     u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
#     c1 = Concatenate()([u1, x4])
#     x5 = conv_block(c1, 64, pool=False)

#     u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
#     c2 = Concatenate()([u2, x3])
#     x6 = conv_block(c2, 48, pool=False)

#     u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
#     c3 = Concatenate()([u3, x2])
#     x7 = conv_block(c3, 32, pool=False)

#     u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
#     c4 = Concatenate()([u4, x1])
#     x8 = conv_block(c4, 16, pool=False)

#     """ Output layer """
#     output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

#     return Model(inputs, output)

## Load model and compile

In [54]:
# importing libraries
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from segmentation_models.metrics import iou_score
import datetime, os

""" Defining Hyperparameters """
img_shape = (480, 480, 3)
num_classes = 4
lr = 5e-5
batch_size = 16
epochs = 10
# Parameters for VGG16 Pretrained Model
BACKBONE = 'vgg16'
activation = 'softmax'


""" Model building and compiling """
# Metrics for VGG16 Pretrained Model
metrics = [sm.metrics.IOUScore(threshold=0.5)]

# Building a UNet with VGG16 as Backbone using segmetation_models library
model = sm.Unet(backbone_name = BACKBONE, 
                input_shape = img_shape, 
                classes = num_classes, 
                activation = activation,
                encoder_weights = 'imagenet')
model.summary()

# Using RMSprop optimizer for stabilizing the val_iou_score in each epoch. 
model.compile(loss = 'categorical_crossentropy', 
              optimizer = tf.keras.optimizers.RMSprop(lr), 
              metrics = metrics)

train_steps = len(X_train)//batch_size
valid_steps = len(X_test)//batch_size

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 480, 480, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 480, 480, 64) 1792        input_12[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 480, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 240, 240, 64) 0           block1_conv2[0][0]               
___________________________________________________________________________________________

## Train model

In [55]:
'''model.fit is used to train the model'''
model_history = model.fit(train_dataset,
        epochs = epochs,
        steps_per_epoch = train_steps,
        validation_data = valid_dataset,
        validation_steps = valid_steps
)

Epoch 1/10
488/488 [==============================] - 397s 807ms/step - loss: 0.2731 - iou_score: 0.2180 - val_loss: 0.1102 - val_iou_score: 0.3769
Epoch 2/10
488/488 [==============================] - 392s 804ms/step - loss: 0.0596 - iou_score: 0.7209 - val_loss: 0.0288 - val_iou_score: 0.6923
Epoch 3/10
488/488 [==============================] - 393s 804ms/step - loss: 0.0204 - iou_score: 0.9309 - val_loss: 0.0120 - val_iou_score: 0.9401
Epoch 4/10
488/488 [==============================] - 391s 802ms/step - loss: 0.0117 - iou_score: 0.9404 - val_loss: 0.0110 - val_iou_score: 0.9401
Epoch 5/10
488/488 [==============================] - 391s 802ms/step - loss: 0.0082 - iou_score: 0.9397 - val_loss: 0.0055 - val_iou_score: 0.9398
Epoch 6/10
488/488 [==============================] - 391s 802ms/step - loss: 0.0055 - iou_score: 0.9401 - val_loss: 0.0051 - val_iou_score: 0.9398
Epoch 7/10
488/488 [==============================] - 392s 804ms/step - loss: 0.0045 - iou_score: 0.9400 - val_l

In [56]:
print("Final Model History")
print(model_history.history)

Final Model History
{'loss': [0.2730637490749359, 0.05958528816699982, 0.02043072134256363, 0.011736622080206871, 0.008204606361687183, 0.005505457986146212, 0.004455679561942816, 0.005366102326661348, 0.003807353088632226, 0.0028437115252017975], 'iou_score': [0.21798183023929596, 0.7209295630455017, 0.9309038519859314, 0.940369188785553, 0.9396529197692871, 0.9401271939277649, 0.940005362033844, 0.9400326609611511, 0.9400274157524109, 0.9400460720062256], 'val_loss': [0.11020644754171371, 0.028828931972384453, 0.011978352442383766, 0.011032418347895145, 0.005451821256428957, 0.005057929549366236, 0.00495494669303298, 0.005906953476369381, 0.003229364985600114, 0.0038589152973145247], 'val_iou_score': [0.3768904209136963, 0.6922722458839417, 0.9400937557220459, 0.9401147365570068, 0.9397609829902649, 0.9397558569908142, 0.9397373795509338, 0.9397491216659546, 0.9397779703140259, 0.9397419691085815]}


## [IMPORTANT] Paste you final model training history here in the markdown.(just double click this line, and you'll be able to edit it. 

NOTE: If we find that your actual model score and what you paste here is differing, your assignment will get rejected.  

Final Model History  
{'loss': [0.2730637490749359, 0.05958528816699982, 0.02043072134256363, 0.011736622080206871, 0.008204606361687183, 0.005505457986146212, 0.004455679561942816, 0.005366102326661348, 0.003807353088632226, 0.0028437115252017975], 'iou_score': [0.21798183023929596, 0.7209295630455017, 0.9309038519859314, 0.940369188785553, 0.9396529197692871, 0.9401271939277649, 0.940005362033844, 0.9400326609611511, 0.9400274157524109, 0.9400460720062256], 'val_loss': [0.11020644754171371, 0.028828931972384453, 0.011978352442383766, 0.011032418347895145, 0.005451821256428957, 0.005057929549366236, 0.00495494669303298, 0.005906953476369381, 0.003229364985600114, 0.0038589152973145247], 'val_iou_score': [0.3768904209136963, 0.6922722458839417, 0.9400937557220459, 0.9401147365570068, 0.9397609829902649, 0.9397558569908142, 0.9397373795509338, 0.9397491216659546, 0.9397779703140259, 0.9397419691085815]}



# Atempts and Descriptions
| SNo.        | Description                              | Previous val_iou_score | Current val_iou_score |
| ----------- | -----------                              | -----------------------|-----------------------|
| 1           | No Changes made to any parameters!       |      -                 |    0.196758           |
| 2           | Noticed that the original shape of the image is 480 x 720, but the original notebook was resizing the image to 256 x 256.By reducing the size of the image, many features in the image are lost. This is why I changed the height and width of the image to the original image's height and width.| 0.196758 | 0.194496 | 
| 3           | Since the number of epochs is too low, tried increasing the number of epochs to 10 to check for improvement.|0.194496                |    0.199426           |
| 4           | Decreasing the batch size to 8 in order to have the model train on more images on each epoch. Increased the number of epochs to 20 to check the effect. |  0.199426                |    0.193903           |
| 5           | Since the traditional method of manually creating tensorflow layers and building a UNet seems to result in lower val_iou_scores, we can try incorporate Trasfer learning using the VGG16 pretrained model.Since the VGG16 model accepts only square images as input, input image shape was changed to (480, 480, 3)|      0.193903                 |    0.232177           |
| 6           | Adjusting the learning rate in order to lower the fluctuation in val_iou_score.       |      0.232177                |    0.690185           |
| 7           | Adjusting the batch size to 16 to reduce overfitting.      |      0.690185                 |    0.917532           |
| 8           | Changing the optimizer to RMSprop to try to control the fluctuations in the model's val_iou_score       |      0.917532               |    0.939769           |
| 9           | Trying to decrease the number of epochs to check if model stabilizes at 0.939 val_iou_socre.       |      0.939769                |     0.939742           |


---
# <center> THE END